# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 21 2022 8:30AM,249108,10,SONSB0001992,"Nextsource Biotechnology, LLC",Released
1,Oct 20 2022 4:03PM,249109,19,ADV80006880,"AdvaGen Pharma, Ltd",Released
2,Oct 20 2022 3:27PM,249106,19,ADV80006879,"AdvaGen Pharma, Ltd",Released
3,Oct 20 2022 3:24PM,249105,12,DSO10202022,Doctors Scientific Organica,Released
4,Oct 20 2022 3:18PM,249102,10,0086092492,ISDIN Corporation,Released
5,Oct 20 2022 3:18PM,249102,10,0086092493,ISDIN Corporation,Released
6,Oct 20 2022 3:18PM,249102,10,0086092497,ISDIN Corporation,Released
7,Oct 20 2022 3:18PM,249102,10,0086092495,ISDIN Corporation,Released
8,Oct 20 2022 3:18PM,249102,10,0086092496,ISDIN Corporation,Released
9,Oct 20 2022 3:18PM,249102,10,0086092494,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
18,249102,Released,27
19,249105,Released,1
20,249106,Released,1
21,249108,Released,1
22,249109,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249102,NaN,NaN,27.0
249105,NaN,NaN,1.0
249106,NaN,NaN,1.0
249108,NaN,NaN,1.0
249109,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249040,0.0,1.0,0.0
249070,19.0,19.0,4.0
249072,0.0,0.0,1.0
249076,0.0,0.0,1.0
249079,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249040,0,1,0
249070,19,19,4
249072,0,0,1
249076,0,0,1
249079,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249040,0,1,0
1,249070,19,19,4
2,249072,0,0,1
3,249076,0,0,1
4,249079,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249040,,1,
1,249070,19,19,4
2,249072,,,1
3,249076,,,1
4,249079,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 21 2022 8:30AM,249108,10,"Nextsource Biotechnology, LLC"
1,Oct 20 2022 4:03PM,249109,19,"AdvaGen Pharma, Ltd"
2,Oct 20 2022 3:27PM,249106,19,"AdvaGen Pharma, Ltd"
3,Oct 20 2022 3:24PM,249105,12,Doctors Scientific Organica
4,Oct 20 2022 3:18PM,249102,10,ISDIN Corporation
31,Oct 20 2022 2:51PM,249098,10,ISDIN Corporation
40,Oct 20 2022 2:49PM,249040,21,"NBTY Global, Inc."
41,Oct 20 2022 2:15PM,249094,10,ISDIN Corporation
52,Oct 20 2022 2:04PM,249090,10,Bio-PRF
60,Oct 20 2022 1:38PM,249088,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 21 2022 8:30AM,249108,10,"Nextsource Biotechnology, LLC",,,1
1,Oct 20 2022 4:03PM,249109,19,"AdvaGen Pharma, Ltd",,,1
2,Oct 20 2022 3:27PM,249106,19,"AdvaGen Pharma, Ltd",,,1
3,Oct 20 2022 3:24PM,249105,12,Doctors Scientific Organica,,,1
4,Oct 20 2022 3:18PM,249102,10,ISDIN Corporation,,,27
5,Oct 20 2022 2:51PM,249098,10,ISDIN Corporation,,7,2
6,Oct 20 2022 2:49PM,249040,21,"NBTY Global, Inc.",,1,
7,Oct 20 2022 2:15PM,249094,10,ISDIN Corporation,1,,10
8,Oct 20 2022 2:04PM,249090,10,Bio-PRF,,6,2
9,Oct 20 2022 1:38PM,249088,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 21 2022 8:30AM,249108,10,"Nextsource Biotechnology, LLC",1,,
1,Oct 20 2022 4:03PM,249109,19,"AdvaGen Pharma, Ltd",1,,
2,Oct 20 2022 3:27PM,249106,19,"AdvaGen Pharma, Ltd",1,,
3,Oct 20 2022 3:24PM,249105,12,Doctors Scientific Organica,1,,
4,Oct 20 2022 3:18PM,249102,10,ISDIN Corporation,27,,
5,Oct 20 2022 2:51PM,249098,10,ISDIN Corporation,2,7,
6,Oct 20 2022 2:49PM,249040,21,"NBTY Global, Inc.",,1,
7,Oct 20 2022 2:15PM,249094,10,ISDIN Corporation,10,,1
8,Oct 20 2022 2:04PM,249090,10,Bio-PRF,2,6,
9,Oct 20 2022 1:38PM,249088,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 21 2022 8:30AM,249108,10,"Nextsource Biotechnology, LLC",1,,
1,Oct 20 2022 4:03PM,249109,19,"AdvaGen Pharma, Ltd",1,,
2,Oct 20 2022 3:27PM,249106,19,"AdvaGen Pharma, Ltd",1,,
3,Oct 20 2022 3:24PM,249105,12,Doctors Scientific Organica,1,,
4,Oct 20 2022 3:18PM,249102,10,ISDIN Corporation,27,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 21 2022 8:30AM,249108,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
1,Oct 20 2022 4:03PM,249109,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN
2,Oct 20 2022 3:27PM,249106,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN
3,Oct 20 2022 3:24PM,249105,12,Doctors Scientific Organica,1.0,NaN,NaN
4,Oct 20 2022 3:18PM,249102,10,ISDIN Corporation,27.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 21 2022 8:30AM,249108,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
1,Oct 20 2022 4:03PM,249109,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0
2,Oct 20 2022 3:27PM,249106,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0
3,Oct 20 2022 3:24PM,249105,12,Doctors Scientific Organica,1.0,0.0,0.0
4,Oct 20 2022 3:18PM,249102,10,ISDIN Corporation,27.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1743646,48.0,21.0,2.0
12,249105,1.0,0.0,0.0
15,249070,4.0,19.0,19.0
16,249086,1.0,0.0,0.0
19,498215,2.0,0.0,0.0
21,498128,1.0,1.0,0.0
22,498155,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1743646,48.0,21.0,2.0
1,12,249105,1.0,0.0,0.0
2,15,249070,4.0,19.0,19.0
3,16,249086,1.0,0.0,0.0
4,19,498215,2.0,0.0,0.0
5,21,498128,1.0,1.0,0.0
6,22,498155,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,48.0,21.0,2.0
1,12,1.0,0.0,0.0
2,15,4.0,19.0,19.0
3,16,1.0,0.0,0.0
4,19,2.0,0.0,0.0
5,21,1.0,1.0,0.0
6,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,48.0
1,12,Released,1.0
2,15,Released,4.0
3,16,Released,1.0
4,19,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21,22
Status,,,,,,,
Completed,2.0,0.0,19.0,0.0,0.0,0.0,0.0
Executing,21.0,0.0,19.0,0.0,0.0,1.0,0.0
Released,48.0,1.0,4.0,1.0,2.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21,22
0,Completed,2.0,0.0,19.0,0.0,0.0,0.0,0.0
1,Executing,21.0,0.0,19.0,0.0,0.0,1.0,0.0
2,Released,48.0,1.0,4.0,1.0,2.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21,22
0,Completed,2.0,0.0,19.0,0.0,0.0,0.0,0.0
1,Executing,21.0,0.0,19.0,0.0,0.0,1.0,0.0
2,Released,48.0,1.0,4.0,1.0,2.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()